In [1]:
import numpy as np
import math
from sympy import *
from scipy import integrate
import matplotlib.pyplot as plt
import random
import time

In [2]:
def TDT(b,c):
    if b+c == 0:
        return 0
    else:
        return ((b-c)**2)/(b+c)

In [3]:
def LS(b,c,N): #local sensitivity
    stat = TDT(b,c)
    M = stat; m = stat
    
    if b >= 2:
        v = [TDT(b-2,c), TDT(b-2,c+1), TDT(b-2,c+2)]
        M = max([max(v),M])
        m = min([min(v),m])
    if b >= 1:
        v = [TDT(b-1,c), TDT(b-1,c+1)]
        M = max([max(v),M])
        m = min([min(v),m])
    if b >= 1 and c >= 1:
        v = TDT(b-1,c-1)
        M = max([v,M])
        m = min([v,m])
    if c >= 2:
        v = [TDT(b+2,c-2), TDT(b+1,c-2), TDT(b+2,c-2)]
        M = max([max(v),M])
        m = min([min(v),m])
    if c >= 1:
        v = [TDT(b,c-1), TDT(b+1,c-1)]
        M = max([max(v),M])
        m = min([min(v),m])
    if b+c <= 2*N-2:
        v = [TDT(b+1,c+1), TDT(b,c+2), TDT(b+2,c)]
        M = max([max(v),M])
        m = min([min(v),m])
    if b+c <= 2*N-1:
        v = [TDT(b,c+1), TDT(b+1,c)]
        M = max([max(v),M])
        m = min([min(v),m])
    if b+c <= 2*N-1 and b >= 1:
        v = TDT(b-1,c+2)
        M = max([v,M])
        m = min([v,m])
    if b+c <= 2*N-1 and c >= 1:
        v = TDT(b+2,c-1)
        M = max([v,M])
        m = min([v,m])
    
    return max([M-stat, stat-m])

In [4]:
def d(x,b,c):
    if b >= x[0] and c >= x[1]:
        return math.ceil(((b+c)-(x[0]+x[1]))/2)
    elif b <= x[0] and c <= x[1]:
        return math.ceil(((x[0]+x[1])-(b+c))/2)
    else:
        return math.ceil(max([math.fabs(b-x[0]), math.fabs(c-x[1])])/2)
    
def gd(x,N):
    return min([d(x,0,2*N), d(x,2,2*N-2), d(x,2*N-2,2), d(x,2*N,0)])

def ud(x,N): # T = 6
    sd = 10000
    for c in range(int(N/4)):
        s = 9+7*c
        for b in range(s,2*N-c+1):
            sd = min([d(x,b,c),d(x,b-2,c+2),sd])
    for b in range(int(N/4)):
        s = 9+7*b
        for c in range(s,2*N-b+1):
            sd = min([d(x,b,c),d(x,b+2,c-2),sd])
    return sd

In [5]:
def RT_EM(x,eps,N,m): #exponential mechanism
    start = time.time()
    u = np.zeros(m); v = np.zeros(m)
    eta = 2*(8*(N-1)/N)/eps
    for i in range(m):
        u[i] = TDT(x[i][0],x[i][1])
        v[i] = u[i] + np.random.gumbel(0,eta,1)
    a = np.argmax(v)
    end = time.time()
    return end-start

def RT_PF(x,eps,N,m): #permute-and-flip
    start = time.time()
    u = np.zeros(m); v = np.zeros(m)
    beta = 2*(8*(N-1)/N)/eps
    for i in range(m):
        u[i] = TDT(x[i][0],x[i][1])
        v[i] = u[i] + np.random.exponential(beta,1)
    a = np.argmax(v)
    end = time.time()
    return end-start

In [6]:
def RT_SPS_g(x,eps,N,m,gamma): #Smooth Private Selection, gd(x) #N = 150, gamma = 2
    start = time.time()
    GS = 8*(N-1)/N; lbeta = math.log(GS/LS(0,299,N))/gd([0,299],N)
    alpha = eps/(2*((gamma-1)**((gamma-1)/gamma)))
    beta = eps/(2*(gamma-1)); lbeta = min(lbeta,beta/m)
    k = 1 - m*lbeta/(2*beta)
    u = np.zeros(m); v = np.zeros(m)
    s = np.zeros(m)
    
    for i in range(m):
        u[i] = TDT(x[i][0],x[i][1])
        if gd(x[i],N) == 0:
            s[i] = GS
        else:
            s[i] = GS*math.exp(-lbeta*gd(x[i],N))
            
    S = max(s)
    for i in range(m):
        v[i] = u[i] + (S/(k*alpha))*math.fabs(np.random.standard_cauchy(1))
    
    a = np.argmax(v)
    end = time.time()
    return end-start

def RT_SPS_u(x,eps,N,m,gamma): #Smooth Private Selection, ud(x) #N = 150, gamma = 2
    start = time.time()
    GS = 8*(N-1)/N; lbeta = math.log(GS/LS(91,209,N))/ud([91,209],N)
    alpha = eps/(2*((gamma-1)**((gamma-1)/gamma)))
    beta = eps/(2*(gamma-1)); lbeta = min(lbeta,beta/m)
    k = 1 - m*lbeta/(2*beta)
    u = np.zeros(m); v = np.zeros(m)
    s = np.zeros(m)
    
    for i in range(m):
        u[i] = TDT(x[i][0],x[i][1])
        if LS(x[i][0],x[i][1],N) > 6:
            s[i] = GS
        else:
            s[i] = GS*math.exp(-lbeta*ud(x[i],N))
            
    S = max(s)
    for i in range(m):
        v[i] = u[i] + (S/(k*alpha))*math.fabs(np.random.standard_cauchy(1))
    
    a = np.argmax(v)
    end = time.time()
    return end-start

In [7]:
def generateData(N):
    d = np.zeros(2)
    s = np.random.binomial(int(2*N),2/3)
    d[0] = np.random.binomial(s,1/2)
    d[1] = s-d[0]
    return d

In [8]:
m = 20
N = 150; eps = 10; gamma = 2

x = np.zeros((m,2)); RT = np.zeros(4)

for j in range(10):
    for i in range(m):
        x[i] = generateData(N)
        
    RT[0] += RT_EM(x,eps,N,m)
    RT[1] += RT_PF(x,eps,N,m)
    RT[2] += RT_SPS_g(x,eps,N,m,gamma)
    RT[3] += RT_SPS_u(x,eps,N,m,gamma)

print(RT/10)

[1.53779984e-04 1.27553940e-04 4.64534760e-04 9.02041411e-01]


In [9]:
m = 50
N = 150; eps = 10; gamma = 2

x = np.zeros((m,2)); RT = np.zeros(4)

for j in range(10):
    for i in range(m):
        x[i] = generateData(N)
        
    RT[0] += RT_EM(x,eps,N,m)
    RT[1] += RT_PF(x,eps,N,m)
    RT[2] += RT_SPS_g(x,eps,N,m,gamma)
    RT[3] += RT_SPS_u(x,eps,N,m,gamma)

print(RT/10)

[3.49736214e-04 3.17740440e-04 1.11136436e-03 2.22536867e+00]


In [10]:
m = 100
N = 150; eps = 10; gamma = 2

x = np.zeros((m,2)); RT = np.zeros(4)

for j in range(10):
    for i in range(m):
        x[i] = generateData(N)
        
    RT[0] += RT_EM(x,eps,N,m)
    RT[1] += RT_PF(x,eps,N,m)
    RT[2] += RT_SPS_g(x,eps,N,m,gamma)
    RT[3] += RT_SPS_u(x,eps,N,m,gamma)

print(RT/10)

[6.58416748e-04 6.36601448e-04 2.27437019e-03 4.59477887e+00]


In [11]:
m = 200
N = 150; eps = 10; gamma = 2

x = np.zeros((m,2)); RT = np.zeros(4)

for j in range(10):
    for i in range(m):
        x[i] = generateData(N)
        
    RT[0] += RT_EM(x,eps,N,m)
    RT[1] += RT_PF(x,eps,N,m)
    RT[2] += RT_SPS_g(x,eps,N,m,gamma)
    RT[3] += RT_SPS_u(x,eps,N,m,gamma)

print(RT/10)

[1.52311325e-03 1.37917995e-03 4.82895374e-03 1.00294551e+01]


In [12]:
m = 500
N = 150; eps = 10; gamma = 2

x = np.zeros((m,2)); RT = np.zeros(4)

for j in range(10):
    for i in range(m):
        x[i] = generateData(N)
        
    RT[0] += RT_EM(x,eps,N,m)
    RT[1] += RT_PF(x,eps,N,m)
    RT[2] += RT_SPS_g(x,eps,N,m,gamma)
    RT[3] += RT_SPS_u(x,eps,N,m,gamma)

print(RT/10)

[3.35624218e-03 3.21173668e-03 1.05774641e-02 2.26949943e+01]


In [13]:
m = 1000
N = 150; eps = 10; gamma = 2

x = np.zeros((m,2)); RT = np.zeros(4)

for j in range(10):
    for i in range(m):
        x[i] = generateData(N)
        
    RT[0] += RT_EM(x,eps,N,m)
    RT[1] += RT_PF(x,eps,N,m)
    RT[2] += RT_SPS_g(x,eps,N,m,gamma)
    RT[3] += RT_SPS_u(x,eps,N,m,gamma)

print(RT/10)

[6.35015965e-03 6.27155304e-03 2.00849295e-02 4.30686604e+01]
